In [100]:
import pandas as pd
import numpy as np
import re
import plotly.express as px

# Copy and paste URL for meeting from CME FedWatch
Links here: https://www.cmegroup.com/markets/interest-rates/cme-fedwatch-tool.html?redirect=/trading/interest-rates/countdown-to-fomc.html
### Copy/paste URL from month meeting to be analyzed into URL below

In [101]:
url="https://cmegroup-tools.quikstrike.net/User/Export/FedWatch/MeetingExport.aspx?MeetingDate=20231213&insid=103276906&qsid=e02905fa-665f-43d1-bef9-5ed34a4ba06c"
df = pd.read_csv(url)

In [102]:
# Pull the meeting date from the URL
meeting_date_match = re.search(r"MeetingDate=(\d+)", url)

if meeting_date_match:
    meeting_date = meeting_date_match.group(1)
    print("Meeting Date:", meeting_date)
else:
    print("Meeting Date not found in the URL")

Meeting Date: 20231213


In [103]:
# Take original dataframe, reindex for date and fillna
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index("Date").fillna(0)

In [104]:
# Get the start and end values of each range
column_titles = df.columns
new_df = pd.DataFrame(column_titles)

# Clean the dataframe
new_df = new_df.rename(columns={0:"range"})
new_df["range"] = new_df["range"].str.replace("(","")
new_df["range"] = new_df["range"].str.replace(")","")

# Add low and high end of target range, and midpoint of range
new_df["start_values"] = new_df["range"].str.split('-').str.get(0).astype(float)
new_df["end_values"] = new_df["range"].str.split('-').str.get(1).astype(float)
new_df["midpoint"] = (new_df["start_values"] + new_df["end_values"])/2

# Transpose the dataframe and pull only midpoint
new_df = new_df.T
midpoint = new_df.tail(1)
midpoint = midpoint.iloc[[0]]

<ipython-input-104-32007b4145cc>:7: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.

<ipython-input-104-32007b4145cc>:8: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.



In [105]:
# Change dataframe column names into the fed funds rate midpoint
df.columns = midpoint.iloc[0]

# Multiply the column names (Midpoints) by the probabilities in each row
result_df = df.mul(df.columns)
result_df = pd.DataFrame(result_df.sum(axis=1)/100).reset_index().round(2)
result_df.columns = ["Date","Median fed funds rate"]

In [106]:
result_df

,Date,Median fed funds rate
0,2022-08-30,3.60
1,2022-08-31,3.58
2,2022-09-01,3.65
3,2022-09-02,3.50
4,2022-09-06,3.63
...,...,...
247,2023-08-23,5.46
248,2023-08-24,5.49
249,2023-08-25,5.53
250,2023-08-28,5.54


In [107]:
fig = px.line(result_df,x="Date", y="Median fed funds rate", title=("Meeting Date: "+ meeting_date))
fig.show()

# Max all time

In [108]:
result_df[result_df["Median fed funds rate"] == result_df["Median fed funds rate"].max()]

,Date,Median fed funds rate
130,2023-03-08,5.62


# Max last 30 days

In [109]:
last30 = result_df[-30:]
last30[last30["Median fed funds rate"] == last30["Median fed funds rate"].max()]

,Date,Median fed funds rate
250,2023-08-28,5.54


# Min all time

In [110]:
result_df[result_df["Median fed funds rate"] == result_df["Median fed funds rate"].min()]

,Date,Median fed funds rate
3,2022-09-02,3.5


# Min last 30 days

In [111]:
last30 = result_df[-30:]
last30[last30["Median fed funds rate"] == last30["Median fed funds rate"].min()]

,Date,Median fed funds rate
222,2023-07-19,5.4
